<a href="https://colab.research.google.com/github/nagadeepthitangudu/Machine-Learning-Projects/blob/master/Resume_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install docx
!pip install PyPDF2
!pip install python-docx
!pip install pdfplumber
!pip install nltk
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docx: filename=docx-0.2.4-py3-none-any.whl size=53898 sha256=2ec1ea9ede6c6d3db2e9044bca1545267e4cb3e0cdd6e17509d24427c4c5bd9d
  Stored in directory: /root/.cache/pip/wheels/81/f5/1d/e09ba2c1907a43a4146d1189ae4733ca1a3bfe27ee39507767
Successfully built docx
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 23.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184489 sha256=696c8920087176cb064c12f976e27a544f133ee340b85b48329b699e62ddcca8
  Stored in directory: /root/.cache/pip/wheels/80/27/06/837436d4c3bd989b957a91679966f207bfd71d358d63a8194d
Successfully built python-docx
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.5/47.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
import glob
import re
import os
from docx import Document
import numpy as np
import PyPDF2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
from joblib import Parallel, delayed
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def read_docx(file_path):
    doc = Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)
def read_pdf(file_path):
    pdf_file = open(file_path, 'rb')
    pdf_reader = PyPDF2.PdfFileReader(pdf_file)
    full_text = []
    for page_num in range(pdf_reader.numPages):
        page = pdf_reader.getPage(page_num)
        full_text.append(page.extractText())
    return '\n'.join(full_text)

In [ ]:
resume_folder = '/content/sample_data/Resumes'
resume_files = os.listdir(resume_folder)
resume_texts = []
resume_ids=[]
start_id= 101
for i, file  in enumerate(resume_files):
    resume_id=start_id+i
    file_path = os.path.join(resume_folder, file)
    if file.endswith('.docx'):
        text = read_docx(file_path)
    elif file.endswith('.pdf'):
        text = read_pdf(file_path)
    resume_ids.append(resume_id)
    resume_texts.append(text)
df_resume = pd.DataFrame({'resume_id':  resume_ids,'text': resume_texts})
df_resume.drop_duplicates(inplace=True)
def extract_keywords(text):
    tfidf_vectorizer = TfidfVectorizer(stop_words='english', use_idf=True, smooth_idf=True, max_features=10)
    tfidf_matrix = tfidf_vectorizer.fit_transform([text])
    feature_names = tfidf_vectorizer.get_feature_names_out()
    keywords = [feature_names[idx] for idx in tfidf_matrix.indices]
    return ", ".join(keywords)  # Join the extracted keywords with commas

In [ ]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
def clean_resume_text(text):
    cleaned_text = re.sub(r'\n|\t', ' ', text).strip()
    cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', '', cleaned_text)
    cleaned_text = cleaned_text.lower()
    cleaned_text = re.sub(r'\S+@\S+', '', cleaned_text)
    cleaned_text = re.sub(r'\d{3} \d{3}\d{4}', '', cleaned_text)
    words = word_tokenize(cleaned_text)
    cleaned_words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    cleaned_text = ' '.join(cleaned_words)
    return cleaned_text

df_resume['text'] = df_resume['text'].apply(clean_resume_text)
df_resume['keywords']=df_resume['text'].apply(extract_keywords)
df_resume.drop_duplicates(inplace=True)
df_resume.head(35)

,resume_id,text,keywords
0,101,ajay kumar csm emailskype ajaydtgmailcom mob 1...,"requirement, tool, case, client, test, team, m..."
1,102,profile summary 11 year work experience progra...,"solution, team, planning, client, scrum, busin..."
2,103,name abiral pandey email phone 9402423303 curr...,"used, database, spring, framework, like, servi..."
3,104,avanthi katta sr business analyst ph 916282925...,"user, test, project, case, rational, document,..."
4,105,summary 13 year total professional experience ...,"requirement, client, team, management, report,..."
5,106,adhi gopalam adhigopalamgmailcom 2812123592 ce...,"test, sprint, team, process, product, manageme..."
6,107,achyuth 5409998048 achyuthjava88gmailcom objec...,"developed, used, database, apache, spring, ser..."
7,108,amulya komatineni 5153091612 8 year experience...,"data, java, used, hibernate, spring, applicati..."
8,109,b shaker shaker0516gmailcom 7142009052 introdu...,"user, process, sprint, team, requirement, proj..."
9,110,summary certified scrum master certified agile...,"customer, client, technology, java, responsibl..."


In [ ]:
df_job=pd.read_csv('/content/Job_descriptn.csv')
df_job.drop_duplicates(inplace=True)
num_jobs = len(df_job)
job_ids = range(1, num_jobs + 1)
df_job['job_id'] = job_ids
df_job['description'] = df_job['Description'].apply(clean_resume_text)
df_job = df_job[['job_id', 'Job Category', 'description']]
df_job['keywords']=df_job['description'].apply(extract_keywords)
df_job

,job_id,Job Category,description,keywords
0,1,Agile Project Manager,project manager specializes leading managing p...,"collaboration, development, iterative, ensurin..."
1,2,Technical Project Manager,project manager strong technical background ov...,"deliver, adhere, deadline, meet, ensuring, coo..."
2,3,Product Manager,responsible developing managing product produc...,"bring, development, collaborating, market, con..."
3,4,Delivery Manager,oversees endtoend delivery project ensuring co...,"client, manages, quality, meet, completed, ens..."
4,5,Offshore/Onsite Coordinator,coordinate activity communication offshore ons...,"execution, allocation, effective, collaboratio..."
5,6,Cloud Solutions Manager,manages cloudbased solution service assessing ...,"organization, adoption, driving, security, sca..."
6,7,Quality Assurance Manager,lead team responsible testing ensuring quality...,"process, assurance, implementing, defining, ap..."
7,8,Test Manager,manages testing process team defines testing s...,"application, reliability, quality, ensures, pl..."
8,9,Global Team Manager,manages coordinate team located different geog...,"border, delivery, collaboration, communication..."
9,10,EDI Analyst,focus electronic data interchange edi transact...,"different, exchange, facilitate, implementing,..."


In [ ]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english', use_idf=True, smooth_idf=True)
tfidf_matrix_job = tfidf_vectorizer.fit_transform(df_job['description'])
tfidf_matrix_resume = tfidf_vectorizer.transform(df_resume['text'])
if tfidf_matrix_resume.shape[1] != tfidf_matrix_job.shape[1]:
    raise ValueError("The number of features in the TF-IDF matrices for job descriptions and resumes don't match.")
print("Job Descriptions:")
print(df_job[['job_id', 'Job Category']])
print("\nResume TF-IDF Matrix:")
print(tfidf_matrix_resume.toarray())
print('\nJob TFIDF_Matrix_Job:')
print(tfidf_matrix_job.toarray())

Job Descriptions:
    job_id                  Job Category
0        1         Agile Project Manager
1        2     Technical Project Manager
2        3               Product Manager
3        4              Delivery Manager
4        5   Offshore/Onsite Coordinator
5        6       Cloud Solutions Manager
6        7     Quality Assurance Manager
7        8                  Test Manager
8        9           Global Team Manager
9       10                   EDI Analyst
10      11   Healthcare Business Analyst
11      12     Medicare/Medicaid Analyst
12      13    EDI Implementation Analyst
13      14                Claims Analyst
14      15      Business Process Modeler
15      16  Construction Project Manager
16      17     Insurance Project Manager
17      18     Six Sigma Project Manager
18      19            IT Project Manager
19      20           Business Strategist
20      21                  Data Analyst

Resume TF-IDF Matrix:
[[0.         0.01323796 0.05829148 ... 0.         0.     

In [ ]:
cosine_similarities = np.dot(tfidf_matrix_job, tfidf_matrix_resume.T).toarray()
print("\nCosine Similarities:")
print(cosine_similarities)


Cosine Similarities:
[[0.29149257 0.24708324 0.21012674 0.12894585 0.32336031 0.38957597
  0.21892481 0.24398416 0.18540146 0.30626469 0.23135185 0.13860053
  0.45047254 0.22013098 0.19605435 0.4139642  0.21654133 0.19860372
  0.20456435 0.25316422 0.23154628 0.30291316]
 [0.23594961 0.1647757  0.03790598 0.11453644 0.21640002 0.26386638
  0.05058057 0.0376348  0.1324667  0.33507748 0.07003244 0.09028802
  0.28964127 0.14628098 0.0504518  0.31711096 0.14819267 0.03506836
  0.04222564 0.19748411 0.0370221  0.24242839]
 [0.14677568 0.30978292 0.03084462 0.06078796 0.10923909 0.28107978
  0.04578024 0.08346339 0.1492753  0.24067448 0.06043519 0.09231897
  0.04107602 0.16252344 0.06395178 0.17607421 0.17916404 0.04865391
  0.06357916 0.11963562 0.05594982 0.08189895]
 [0.20407559 0.30676128 0.020771   0.11413899 0.31554735 0.18464093
  0.08779259 0.06968285 0.09818023 0.26289508 0.05168903 0.07704584
  0.22506045 0.1414874  0.05814219 0.29369512 0.14333644 0.07212834
  0.07046007 0.142648

In [ ]:
recommended_resumes = []
for i in range(len(df_job)):
    top_indices = cosine_similarities[i].argsort()[-5:][::-1]  # Get the indices of the top 5 recommendations
    top_resume_ids = df_resume.iloc[top_indices]['resume_id'].tolist()
    job_category = df_job.iloc[i]['Job Category']
    recommended_resumes.append({'Job Category': job_category, 'Recommended Resumes': top_resume_ids})

recommended_resumes_df = pd.DataFrame(recommended_resumes)
print("\nRecommended Resumes:")
print(recommended_resumes_df)


Recommended Resumes:
                    Job Category        Recommended Resumes
0          Agile Project Manager  [113, 116, 106, 105, 110]
1      Technical Project Manager  [110, 116, 113, 106, 122]
2                Product Manager  [102, 106, 110, 117, 116]
3               Delivery Manager  [105, 102, 116, 110, 122]
4    Offshore/Onsite Coordinator  [106, 116, 122, 110, 113]
5        Cloud Solutions Manager  [107, 103, 108, 115, 111]
6      Quality Assurance Manager  [101, 120, 115, 106, 104]
7                   Test Manager  [101, 120, 104, 112, 115]
8            Global Team Manager  [106, 110, 113, 116, 122]
9                    EDI Analyst  [112, 109, 108, 119, 118]
10   Healthcare Business Analyst  [117, 114, 104, 120, 109]
11     Medicare/Medicaid Analyst  [112, 109, 118, 119, 111]
12    EDI Implementation Analyst  [120, 101, 106, 116, 104]
13                Claims Analyst  [102, 104, 116, 120, 109]
14      Business Process Modeler  [103, 109, 117, 114, 119]
15  Construction P